# SVM algorithm

In [70]:
import numpy as np 
import pandas as pd 

In [71]:
# load train and test data
trainDataFilePath = "C:/Users/Yutong Liu/Videos/data/processed/train.json"
testDataFilePath = "C:/Users/Yutong Liu/Videos/data/processed/test.json"

trainData = pd.read_json(trainDataFilePath)
testData = pd.read_json(testDataFilePath)

In [73]:
import matplotlib.pyplot as plt
from skimage.feature import hog
# Histogram of Oriented Gradients 
def HOG(image):
    img1 = image[:,:,0]
    img2 = image[:,:,1]
    #img3 = image[:,:,2]
    hog_image = np.zeros(image.shape)
    fd, hog_image[:,:,0] = hog(img1, orientations=8, pixels_per_cell=(10, 10),
                    cells_per_block=(1, 1),block_norm='L2',visualise=True)
    fd, hog_image[:,:,1] = hog(img2, orientations=8, pixels_per_cell=(10, 10),
                    cells_per_block=(1, 1),block_norm='L2',visualise=True)
    #fd, hog_image[:,:,2] = hog(img3, orientations=8, pixels_per_cell=(10, 10),
                    #cells_per_block=(1, 1),block_norm='L2',visualise=True)
    return hog_image
# Normalize the image where the components are between 0 and 1
def norm(image):
    img1 = image[:,:,0]
    img2 = image[:,:,1]
    norm_image = np.zeros(image.shape)
    norm_image[:,:,0] = (img1 - img1.min())/(img1.max()-img1.min())
    norm_image[:,:,1] = (img2 - img2.min())/(img2.max()-img2.min())
    return norm_image

In [75]:
from sklearn.model_selection import train_test_split
# Process data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in trainData["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in trainData["band_2"]])
x_band3 = (x_band1 + x_band2)/2
# ,x_band3[:, :, :, np.newaxis]
X_train = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
y_train = np.array(trainData["is_iceberg"])

In [89]:
from sklearn.model_selection import train_test_split
nsize= X_train.shape[0]
Xnew_train = np.zeros((nsize,75*75*4))
for i in range (nsize):
    img = X_train[i,:,:,:]
    img1 = HOG(img).reshape((1,75*75*2))
    img2 = norm(img).reshape((1,75*75*2))
    img = np.hstack((img1,img2))
    Xnew_train[i,:] = img
    
# shuffle data and separate it into train group and test group
Xtr,Xts,ytr,yts = train_test_split(Xnew_train,y_train,test_size=0.2,random_state=None)
print("Xtrain:", Xtr.shape)
print("Xtest:", Xts.shape)

Xtrain: (1283, 22500)
Xtest: (321, 22500)


In [90]:
from sklearn import svm
model = svm.SVC(probability=False,kernel='rbf',C=10,gamma=0.01)
model.fit(Xtr,ytr)
yhat = model.predict(Xts)
acc = np.mean(yhat==yts)
print('Accuracy =', acc)

Accuracy = 0.86292834891
